In [5]:
import clickhouse_connect
from splinkclickhouse import ClickhouseAPI
import splink.comparison_library as cl
from splink import Linker, SettingsCreator, block_on
import pandas as pd
from sqlalchemy import create_engine


SQLALCHEMY_DATABASE_URL = "clickhouse://default:@10.0.1.80/default"


engine = create_engine(SQLALCHEMY_DATABASE_URL)
query = "SELECT * FROM table_dataset1 TabSeparatedWithNamesAndTypes"
df = pd.read_sql(query, engine)

In [6]:
ch = clickhouse_connect.get_client(dsn=SQLALCHEMY_DATABASE_URL)
db_api = ClickhouseAPI(ch)
settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.JaroWinklerAtThresholds("first_name"),
        cl.JaroAtThresholds("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.DamerauLevenshteinAtThresholds("city").configure(term_frequency_adjustments=True),
        cl.JaccardAtThresholds("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "dob"),
        block_on("surname"),
    ],
)

linker = Linker(df, settings, db_api=db_api)

SETTINGS VALIDATION: Errors were identified in your settings dictionary. 

Setting: `unique_id_column_name`

       - Missing column(s) from input dataframe(s): `unique_id`

Invalid Columns(s) in Blocking Rule(s)

    SQL: `(l."first_name" = r."first_name") AND (l."dob" = r."dob")`
       - Missing column(s) from input dataframe(s): `dob`, `first_name`

    SQL: `l."surname" = r."surname"`
       - Missing column(s) from input dataframe(s): `surname`

Invalid Columns(s) in Comparison(s)

Comparison: first_name
--------------------------------------
    SQL: `"first_name_l" IS NULL OR "first_name_r" IS NULL`
       - Missing column(s) from input dataframe(s): `first_name`

    SQL: `"first_name_l" = "first_name_r"`
       - Missing column(s) from input dataframe(s): `first_name`

    SQL: `jaroWinklerSimilarity("first_name_l", "first_name_r") >= 0.9`
       - Missing column(s) from input dataframe(s): `first_name`

    SQL: `jaroWinklerSimilarity("first_name_l", "first_name_r") >= 0.7`
